In [1]:
library(MASS)  #NB Regression
library(plotrix) #color scale
require(pscl) #zeroinfl

AllSeg = c('R2U','R4D','R4U','U2U','U4U','U4D','U6U')
AllInt = c('R3ST'  ,'R4ST'  ,'RM3ST'  ,'RM4ST'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG')
Project_DIR  = '//CHCFPP01/Proj/ILDOT/650511SAFETYPROGRAM/4_WorkData/WO00_OtherStates/NewMexico/3_WorkData/7_NetworkScreening/6.Python_Notebook'
setwd(Project_DIR)

Loading required package: pscl
Classes and Methods for R developed in the
Political Science Computational Laboratory
Department of Political Science
Stanford University
Simon Jackman
hurdle and zeroinfl functions by Achim Zeileis


In [2]:
Seg_DF = read.csv(paste0('Segs.csv'))
Int_DF = read.csv(paste0('Ints.csv'))
colnames(Seg_DF)
colnames(Int_DF)
Seg_DF['AADT'] = log(Seg_DF['AADT'])
Seg_DF['TOT_OC'] = Seg_DF['KAB']
Seg_DF['Length'] = log(Seg_DF['Mileage'])
Int_DF['AADT_Major'] = log(Int_DF['AADT_Major'])
Int_DF['AADT_Minor'] = log(Int_DF['AADT_Minor'])
Int_DF['TOT_OC'] = Int_DF['KAB']
Int_DF['FType'] = Int_DF['IFType']
Int_DF[which(is.na(Int_DF$TOT_OC)),'TOT_OC'] = 0
Seg_DF['FID'] = Seg_DF$SegID
Int_DF['FID'] = Int_DF$Int_ID

#Seg_DF = Seg_DF[which(exp(Seg_DF$AADT)>500),]

[1] "SegID"         "ROUTE_ID"      "BEG_POINT"     "END_POINT"    
 [5] "F_SYSTEM"      "FACILITY_TYPE" "OWNERSHIP"     "URBAN_CODE"   
 [9] "THROUGH_LANES" "MEDIAN_TYPE"   "LANE_WIDTH"    "SPEED_LIMIT"  
[13] "AADT"          "LOC_ERROR"     "ORIG_FID"      "Shape_Length" 
[17] "SegID.1"       "FType"         "P3"            "Mileage"      
[21] "Year"          "VMT"           "Killed"        "ClassA"       
[25] "ClassB"        "KAB"

[1] "OBJECTID"        "Int_ID"          "Signal"          "Legs"           
 [5] "RIDList"         "IFType"          "I_URBANCODE"     "FType_Major"    
 [9] "FType_Minor"     "F_SYSTEM_Major"  "F_SYSTEM_Minor"  "OWNERSHIP_Major"
[13] "OWNERSHIP_Minor" "AADT_Major"      "AADT_Minor"      "Year"           
[17] "Killed"          "ClassA"          "ClassB"          "EnteringVeh"    
[21] "StateLocal"      "AADTStatus"      "KAB"             "Count"

In [55]:
M = zeroinfl(TOT_OC ~ AADT_Major + AADT_Minor,data = Int_DF[which(Int_DF$FType=='U3SG'),], dist = "negbin", EM = TRUE,link='logit')
summary(M)


Call:
zeroinfl(formula = TOT_OC ~ AADT_Major + AADT_Minor, data = Int_DF[which(Int_DF$FType == 
    "U3SG"), ], dist = "negbin", link = "logit", EM = TRUE)

Pearson residuals:
    Min      1Q  Median      3Q     Max 
-0.2763 -0.2630 -0.2536 -0.2310 16.1199 

Count model coefficients (negbin with log link):
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  -1.9140     3.6629  -0.523    0.601    
AADT_Major    0.1571     0.3208   0.490    0.624    
AADT_Minor   -0.1163     0.2758  -0.422    0.673    
Log(theta)   -2.2003     0.3536  -6.223 4.87e-10 ***

Zero-inflation model coefficients (binomial with logit link):
            Estimate Std. Error z value Pr(>|z|)
(Intercept)  12.5624    11.6371   1.080    0.280
AADT_Major   -0.4376     0.9542  -0.459    0.647
AADT_Minor   -1.1888     0.9627  -1.235    0.217
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1 

Theta = 0.1108 
Number of iterations in BFGS optimization: 16 
Log-likelihood: -399.1 on 7 Df

In [56]:
M2 = glm(TOT_OC ~ AADT + offset(Length), data = Seg_DF, family= negative.binomial(theta = 10,link= "log"))
summary(M2)


Call:
glm(formula = TOT_OC ~ AADT + offset(Length), family = negative.binomial(theta = 10, 
    link = "log"), data = Seg_DF)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-3.1724  -0.4875  -0.3099  -0.1708   8.6577  

Coefficients:
            Estimate Std. Error t value Pr(>|t|)    
(Intercept) -6.99929    0.12010  -58.28   <2e-16 ***
AADT         0.59713    0.01435   41.62   <2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for Negative Binomial(10) family taken to be 1.683055)

    Null deviance: 17922  on 29440  degrees of freedom
Residual deviance: 14616  on 29439  degrees of freedom
AIC: 20946

Number of Fisher Scoring iterations: 6


In [3]:
print(Sys.time())
Seg_MD = NULL
for(ft in AllSeg){
    MD = LocalSPF(Seg_DF[which(Seg_DF$FType==ft),],ft, 'GAADT')
    Seg_MD = c(Seg_MD,MD)
    #write.csv(MD[paste0(ft,'_D')],paste0(ft,'_Fitted','.csv'))
    print(ft)
}
print(Sys.time())

[1] "2018-11-06 10:12:52 CST"


ERROR: Error in LocalSPF(Seg_DF[which(Seg_DF$FType == ft), ], ft, "GAADT"): could not find function "LocalSPF"


In [89]:
df = data.frame()
for(i in 1:(length(Seg_MD)/3)){
    M = Seg_MD[3*i-2][[1]]
    D = Seg_MD[3*i-1][[1]]
    Type = toString(unique(D$FType)[1])
    m = summary(M)
    a = m$coefficients
    df = rbind(df,data.frame(Type=Type,Intercept=a[1,1],AADT=a[2,1],Dispersion=m$dispersion))
}
write.csv(df,paste0('Seg_SPFs.csv'))

In [57]:
a = summary(Seg_MD$R2U_M)
a = a$coefficients
print(a[1,1])
print(a[2,1])

[1] -6.840424
[1] 0.5743851


In [36]:
df = SPFTable(Seg_MD,'GAADT')
write.csv(df,'Seg_SPFs.csv')

,Type,SampleSize,Coef,Es,Pv,CV,SE,LB,UB
1.GAADT,R2U,6759,Intercept,-6.8404237,2.474593e-180,0.03386722,0.23166612,-7.29456233,-6.3862852
4.GAADT,R2U,6759,AADT,0.5743851,3.103577e-68,0.05664041,0.03253341,0.51060932,0.6381608
6.GAADT,R4D,376,Intercept,-7.0642815,2.859565e-06,0.21038004,1.48618381,-9.98655461,-4.1420083
9.GAADT,R4D,376,AADT,0.6495046,1.222079e-04,0.25756577,0.16729016,0.32056313,0.9784461
11.GAADT,R4U,1255,Intercept,-7.6128923,2.227740e-20,0.10623631,0.80876559,-9.19957393,-6.0262106
14.GAADT,R4U,1255,AADT,0.6845231,3.061272e-13,0.13566292,0.09286441,0.50233653,0.8667097
16.GAADT,U2U,10339,Intercept,-6.4784419,4.432565e-18,0.11516647,0.74609929,-7.94094080,-5.0159429
19.GAADT,U2U,10339,AADT,0.4221862,3.257007e-06,0.21475539,0.09066677,0.24446182,0.5999106
21.GAADT,U4U,5400,Intercept,-7.5580226,1.199055e-11,0.14716480,1.11227487,-9.73853008,-5.3775152
24.GAADT,U4U,5400,AADT,0.6574907,2.172397e-08,0.17837798,0.11728187,0.42757096,0.8874105


In [62]:
for(ft in AllInt){
    MD = LocalSPF(Int_DF[which(Int_DF$FType==ft),],ft , 'GAADT')
    print(ft)
}

[1] "R3ST"
[1] "R4ST"
[1] "RM3ST"
[1] "RM4ST"
[1] "U3ST"
[1] "U4ST"
[1] "U3SG"
[1] "U4SG"


In [4]:
SPFTable      = function(MDList,SPFType,ConfLevel = 0.95){
  ExportSPF = function(InTable){
    AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
    AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U')
    AllP = NULL;for(Type in c(AllInt,AllSeg)){AllP = c(AllP,ListPredictors(Type))};AllP = c('X.Intercept.',unique(AllP))
    Type = NULL
    Coef = NULL
    SampleSize   = NULL
    Es   = NULL
    SE   = NULL
    Pv   = NULL
    LB   = NULL
    UB   = NULL
    CV   = NULL
    for(r in rownames(InTable)){
      ss = InTable[r,'Sample.Size']
      CL = NULL
      for(n in names(InTable)){
        if(!(n=='Sample.Size')){
          for(p in AllP){
            if(n %in% c(paste(p,'.Es',sep=''),paste(p,'.SE',sep=''),paste(p,'.Pv',sep=''),paste(p,'.UB',sep=''),paste(p,'.LB',sep=''),paste(p,'.CV',sep=''))){
              if(!(p %in% CL)){
                Type = c(Type,r)
                SampleSize = c(SampleSize,ss)
                if(p=="X.Intercept."){Coef=c(Coef,"Intercept")}
                else{Coef=c(Coef,p)}
                CL = c(CL,p)
                cf = p
              }
            }
          }
          Flag = FALSE
          if(paste(cf,'.Es',sep='')==n){Es = c(Es,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.SE',sep='')==n){SE = c(SE,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.Pv',sep='')==n){Pv = c(Pv,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.LB',sep='')==n){LB = c(LB,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.UB',sep='')==n){UB = c(UB,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.CV',sep='')==n){CV = c(CV,InTable[r,n]);Flag = TRUE}
          if(!Flag){
            a=1
          }
        }
      }
    }
    D = data.frame(Type,SampleSize,Coef,Es,Pv,CV,SE,LB,UB)
    Out = NULL
    for(i in 1:length(D[,1])){
      if(sum(D[i,4:9])==0){
        Out = rbind(Out,D[i,])
      }
    }
    D = D[!(rownames(D)  %in% rownames(Out)) ,]
    return(D)  
  }
  TAll = data.frame()
  for(i in 1:(length(MDList)/3)){
    M = MDList[3*i-2][[1]]
    D = MDList[3*i-1][[1]]
    TAll = rbind(TAll,CoefSummary(list('Model'=M,'Data'=D)))
  }
  Out = ExportSPF(TAll)
  rownames(Out) = paste(rownames(Out),SPFType,sep='.')
  return(Out)
}  
SPFTable2      = function(MDList,SPFType,ConfLevel = 0.95){
  ExportSPF = function(InTable){
    AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
    AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U')
    AllP = NULL;for(Type in c(AllInt,AllSeg)){AllP = c(AllP,ListPredictors(Type))};AllP = c('X.Intercept.',unique(AllP))
    Type = NULL
    Coef = NULL
    SampleSize   = NULL
    Es   = NULL
    SE   = NULL
    Pv   = NULL
    LB   = NULL
    UB   = NULL
    CV   = NULL
    for(r in rownames(InTable)){
      ss = InTable[r,'Sample.Size']
      CL = NULL
      for(n in names(InTable)){
        if(!(n=='Sample.Size')){
          for(p in AllP){
            if(n %in% c(paste(p,'.Es',sep=''),paste(p,'.SE',sep=''),paste(p,'.Pv',sep=''),paste(p,'.UB',sep=''),paste(p,'.LB',sep=''),paste(p,'.CV',sep=''))){
              if(!(p %in% CL)){
                Type = c(Type,r)
                SampleSize = c(SampleSize,ss)
                if(p=="X.Intercept."){Coef=c(Coef,"Intercept")}
                else{Coef=c(Coef,p)}
                CL = c(CL,p)
                cf = p
              }
            }
          }
          Flag = FALSE
          if(paste(cf,'.Es',sep='')==n){Es = c(Es,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.SE',sep='')==n){SE = c(SE,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.Pv',sep='')==n){Pv = c(Pv,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.LB',sep='')==n){LB = c(LB,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.UB',sep='')==n){UB = c(UB,InTable[r,n]);Flag = TRUE}
          if(paste(cf,'.CV',sep='')==n){CV = c(CV,InTable[r,n]);Flag = TRUE}
          if(!Flag){
            a=1
          }
        }
      }
    }
    D = data.frame(Type,SampleSize,Coef,Es,Pv,CV,SE,LB,UB)
    Out = NULL
    for(i in 1:length(D[,1])){
      if(sum(D[i,4:9])==0){
        Out = rbind(Out,D[i,])
      }
    }
    D = D[!(rownames(D)  %in% rownames(Out)) ,]
    return(D)  
  }
  TAll = data.frame()
  for(i in 1:(length(MDList)/3)){
    M = MDList[3*i-2][[1]]
    D = MDList[3*i-1][[1]]
    TAll = rbind(TAll,CoefSummary(list('Model'=M,'Data'=D)))
  }
  Out = ExportSPF(TAll)
  rownames(Out) = paste(rownames(Out),SPFType,sep='.')
  return(Out)
}  


In [5]:
FilterData      = function(Type,Field){
  # c('Cook','JR','Lev','AADT Major','AADT Minor')
  R3ST  = c(c(3,1,2),c(12500,10000 ))
  R4ST  = c(c(2,0,3),c(15000,6000 ))
  RM3ST = c(c(1,1,0),c(30000,8000 ))
  RM4ST = c(c(1,0,0),c(20000,4000 ))
  U3ST = c(c(1,0,0),c(50000,30000 ))
  U4ST = c(c(1,0,0),c(30000,15000 ))
  U3SG = c(c(2,0,1),c(60000,35000))
  U4SG = c(c(1,0,2),c(60000,40000))
  R2U = c(c(1,2,0),c(15000,0.1))
  R4U = c(c(1,0,0),c(40000,0.1))
  R4D = c(c(1,1,8),c(22000,0.1))
  U2U = c(c(1,0,0),c(30000,0.1))
  U4U = c(c(1,0,1),c(60000,0.1))
  U4D = c(c(3,0,3),c(40000,0.1))
  U6U = c(c(1,0,1),c(70000,0.1))
  T1        = list(R3ST  ,R4ST  ,RM3ST  ,RM4ST  ,U3ST  ,U4ST  ,U3SG  ,U4SG  ,R2U  ,R4D  ,R4U  ,U2U  ,U4U  ,U4D  ,U6U)
  names(T1) = c(  'R3ST','R4ST','RM3ST','RM4ST','U3ST','U4ST','U3SG','U4SG','R2U','R4D','R4U','U2U','U4U','U4D','U6U')  
  
  if(Field=='Outliers'){Out = T1[Type][[1]][1:3]}
  if(Field=='Volumes' ){Out = T1[Type][[1]][4:5]}
  return(Out)
}

In [6]:
SaveImage = function(FileName,Format){
  FN = FileName
  if(Format=='wmf'){win.metafile(paste(FN,'.wmf',sep=''))}
  if(Format=='jpg'){jpg(filename = paste(FN,'.jpg',sep=''), width = 1500, height = 1000, units = "px", pointsize = 12,quality = 100, res =200,bg='transparent' )}
  if(Format=='png'){png(filename = paste(FN,'.png',sep=''), width = 12   , height = 7   , units = "in", pointsize = 12              , res =1200,bg='transparent' )}
  if(Format=='bmp'){bmp(paste(FN,'.bmp',sep=''))}
}


In [7]:
LocalSPF      = function(D1,Type,SPFType){
  IntTypes = c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG')
  SegTypes = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U','R4F','U4F','U6F')
  AllInt   = c('AllInt','AllIntSig','AllIntStp')
  AllSeg   = c('AllSeg')
  InSPFType = SPFType
  
  #Read Data --> D1
  
  #Apply Volume Ranges --> D2
  VolRange = FilterData(Type,'Volumes')
  D2 = D1
  if(Type %in% c(IntTypes,AllInt)){
    D2     = D1[ (D1$AADT_Major<log(VolRange[1]) & D1$AADT_Minor<log(VolRange[2])),]
    D_Out1 = D1[!(D1$AADT_Major<log(VolRange[1])),]
    D_Out2 = D1[!(D1$AADT_Minor<log(VolRange[2])),]
    D_Out1['AADT_Major'] = exp(D_Out1['AADT_Major']);D_Out1['AADT_Minor'] = exp(D_Out1['AADT_Minor'])
    D_Out2['AADT_Major'] = exp(D_Out2['AADT_Major']);D_Out2['AADT_Minor'] = exp(D_Out2['AADT_Minor'])
    #print((unique(D_Out1[c('FID','AADT_Major')])))
    #print((unique(D_Out2[c('FID','AADT_Minor')])))
  }
  if(Type %in% c(SegTypes,AllSeg)){
    D2     = D1[ (D1$AADT  <log(VolRange[1]) & D1$Length>log(VolRange[2])),]
    D_Out1 = D1[!(D1$AADT  <log(VolRange[1])),]
    D_Out2 = D1[!(D1$Length>log(VolRange[2])),]
    D_Out1['AADT'] = exp(D_Out1['AADT']);D_Out1['Length'] = exp(D_Out1['Length'])
    D_Out2['AADT'] = exp(D_Out2['AADT']);D_Out2['Length'] = exp(D_Out2['Length'])
    #print((unique(D_Out1[c('FID','AADT')])))
  }
  #print(paste('Volume Ranges: ',length(D1[,1])-length(D2[,1]),' Deleted'))
  #Apply Outliers --> D3
  Os = FilterData(Type,'Outliers')
  MD2   = GLMModels(D2 ,'Base',Type)
  D3    = D2
  if(sum(Os)>0){
    D_Out = Outliers(MD2 ,1,Type ,Os)
    #print(unique(D_Out[c('FID','Outlier','TOT_OC')]))
    D3    = D2[!(rownames(D2)  %in% rownames(D_Out)) ,]}
  
  #State SPF --> D4, MD4
  D4 = D3
  if(SPFType=='Base'){
    D4  = BaseData(D3)
    #print(paste('Base Data: ',length(D3[,1])-length(D4[,1]),' Deleted'))
  }
  if(SPFType=='GAADT'){SPFType = 'Base'}
  MD4 = GLMModels(D4 ,SPFType,Type)
  HSMCoef = HSMValues(Type,1)
  
  HSMFittedb = rep(-1,length(D4[,1]))
  HSMFitted  = rep(-1,length(D3[,1]))
  
  if(Type %in% c(IntTypes,AllInt)){StateFittedb  = GLMPredict(MD4$M, D4, Type)}
  if(Type %in% c(SegTypes,AllSeg)){StateFittedb  = GLMPredict(MD4$M, D4, Type)}
  if('TOT_PC' %in% names(D4)){HSMFittedb = D4$TOT_PC*1}
  if(!('TOT_PC' %in% names(D4))){
    if(Type %in% c(IntTypes,AllInt)){HSMFittedb  = HSMPredict(MD4$M, D4,1)}
    if(Type %in% c(SegTypes,AllSeg)){HSMFittedb  = HSMPredict(MD4$M, D4,1)}
  }
  
  
  if(Type %in% c(IntTypes,AllInt)){StateFitted  = GLMPredict(MD4$M, D3, Type)}
  if(Type %in% c(SegTypes,AllSeg)){StateFitted  = GLMPredict(MD4$M, D3, Type)}
  #if('CCMF' %in% names(D3) & InSPFType == 'Base'){StateFitted = StateFitted * D3$CCMF}
  if('TOT_PC' %in% names(D3)){HSMFitted = D3$TOT_PC*1}
  if(!('TOT_PC' %in% names(D3))){
    if(Type %in% c(IntTypes,AllInt)){HSMFitted  = HSMPredict(MD4, D3,1)}
    if(Type %in% c(SegTypes,AllSeg)){HSMFitted  = HSMPredict(MD4, D3,1)}
  }
  
  MD4$Data['StateFitted'] = StateFittedb
  MD4$Data['HSMFitted'  ] = HSMFittedb
  AllData = D3
  AllData['StateFitted'] = StateFitted
  AllData['HSMFitted'  ] = HSMFitted
  AllData['StudRes'    ] = studres(MD4$M)
  M = paste(Type,'M',sep='_')
  D = paste(Type,'D',sep='_')
  A = paste(Type,'A',sep='_')
  MD4 = list(M=MD4$M,D=MD4$D,A=AllData)
  names(MD4) = c(M,D,A)
  
  #Plots
  PlotXX(D2,D1,VolRange,HSMCoef,Type,F)
  #PlotCF(D3,Type,Years)
  #PlotCFCompare(MD3,Type)
  Plot2D(Type,MD4 ,FALSE,Type ,data.matrix(HSMCoef[Type,]),FALSE)
  CUREPlot(MD4,Type)
  
  #Output
  return(MD4) 
}


In [8]:
PlotXX        = function(D2,D1,VolRange,HSMDefaults,Title,LogScale = FALSE){
  PlotXXI  = function(D2,D1,VolRange,HSMDefaults,Title){
    X1  = exp(D1$AADT_Major)
    X2  = exp(D1$AADT_Minor)
    Xo1 = exp(D1[!(rownames(D1) %in% rownames(D2)),]$AADT_Major)
    Xo2 = exp(D1[!(rownames(D1) %in% rownames(D2)),]$AADT_Minor)
    Xi1 = exp(D2$AADT_Major)
    Xi2 = exp(D2$AADT_Minor)
    x1r = range(0,X1,VolRange[1])
    x2r = range(0,X2,VolRange[2])
    x2r[2] = x2r[2] * 1.4
    layout(1)
    plot.new()
    layout(matrix(c(2,1,4,3),2,2),width=c(6,2),height=c(3,6))
    
    par(mar=c(6.1,6.1,0,0))
    plot(x=X1,y=X2,xlab='AADT Major',ylab='AADT Minor',col=color.scale(D1$TOT_OC,c(0,1,1),0,1),xlim=x1r,ylim=x2r)
    par(new=TRUE)
    plot(x=Xo1,y=Xo2,xlab='',ylab='',col='red',xlim=x1r,ylim=x2r,pch=4)
    grid()
    segments(c(VolRange[1],x1r[1]),c(x2r[1],VolRange[2]),c(VolRange[1],VolRange[1]),c(VolRange[2],VolRange[2]),col='brown',lty=2)
    l1 = paste('Sample Size: ',length(X1), ', Reduced to: ',length(Xi1),sep='')
    l2 = paste('AADT Major Max: ',round((max(Xi1))))
    l3 = paste('AADT Minor Max: ',round((max(Xi2))))
    legend(legend=c(l1,l2,l3),'topright')
    legend(legend='Domain of Applicability',lty=2,col='brown','topleft')
    
    par(mar=c(0,6.1,6.1,0))
    boxplot(X1,xaxt='n',horizontal=TRUE,ylim=x1r,xlim=c(0,2),col='lightblue',border='blue')
    grid(nx=NULL,ny=0)
    axis(3)
    axis(2,at=c(1),label=c('State Data'),las=2)
    par(new=TRUE);plot(y=rep(x1r[2]/2,length(Xo1[Xo1>VolRange[1]])),x=2/(x1r[2]-x1r[1])*(Xo1[Xo1>VolRange[1]]-x1r[1]),xlab='',ylab='',col='red',xlim=c(0,2),ylim=x1r,pch=4,xaxt='n',yaxt='n')
    
    par(mar=c(6.1,0,0,4.1))
    boxplot(X2,yaxt='n',ylim=x2r,xlim=c(0,2),col='lightblue',border='blue')
    grid(ny=NULL,nx=0)
    axis(4)
    axis(1,at=c(1),label=c('State Data'),las=2)
    par(new=TRUE);plot(x=rep(1,length(Xo2[Xo2>VolRange[2]])),y=Xo2[Xo2>VolRange[2]],xlab='',ylab='',col='red',xlim=c(0,2),ylim=x2r,pch=4,xaxt='n',yaxt='n')
    
    title(paste(Title,'Predictor vs Predictor Plot Compared with HSM Ranges'),outer=TRUE,line=-2)
    layout(1)
    par(mar=c(5,4,4,2)+0.1)
  }
  PlotXXS  = function(D2,D1,VolRange,HSMDefaults,Title){
    X1  = exp(D1$AADT)
    X2  = exp(D1$Length)
    Xo1 = exp(D1[!(rownames(D1) %in% rownames(D2)),]$AADT)
    Xo2 = exp(D1[!(rownames(D1) %in% rownames(D2)),]$Length)
    Xi1 = exp(D2$AADT)
    Xi2 = exp(D2$Length)
    #HSMX1 = HSMDefaults[Title,]$AADT_Major_UB
    #HSMX2 = HSMDefaults[Title,]$Length_LB
    x1r = range(0,X1,(VolRange[1]))
    x2r = range(0,X2,(VolRange[2]))
    x2r[2] = x2r[2] * 1.4
    layout(1)
    plot.new()
    layout(matrix(c(2,1,4,3),2,2),width=c(6,2),height=c(3,6))
    
    par(mar=c(6.1,6.1,0,0))
    plot(x=X1,y=X2,xlab='AADT',ylab='Length',col=color.scale(D1$TOT_OC,c(0,1,1),0,1),xlim=x1r,ylim=x2r)
    par(new=TRUE)
    plot(x=Xo1,y=Xo2,xlab='',ylab='',col='red',xlim=x1r,ylim=x2r,pch=4)
    grid()
    segments(c(VolRange[1],x1r[1]),c(x2r[2],VolRange[2]),c(VolRange[1],VolRange[1]),c(VolRange[2],VolRange[2]),col='brown',lty=2)
    l1 = paste('Sample Size: ',length(X1), ', Reduced to: ',length(Xi1),sep='')
    l2 = paste('AADT Max: ',round((max(Xi1))))
    l3 = paste('Length Min: ',round((min(Xi2))*100)/100)
    legend(legend=c(l1,l2,l3),'topright')
    legend(legend='Domain of Applicability',lty=2,col='brown','topleft')
    
    par(mar=c(0,6.1,6.1,0))
    boxplot(X1,xaxt='n',horizontal=TRUE,ylim=x1r,xlim=c(0,2),col='lightblue',border='blue')
    grid(nx=NULL,ny=0)
    #segments(0,2,HSMX1,2,col='red')
    #par(new=TRUE);plot(y=c(2,2),x=c(0,HSMX1),ylim=c(0,2),xlim=x1r,pch=10,xaxt='n',yaxt='n',ylab='',col='red',xlab='')
    axis(3)
    axis(2,at=c(1),label=c('State Data'),las=2)
    par(new=TRUE);plot(y=rep(x1r[2]/2,length(Xo1[Xo1>VolRange[1]])),x=3/(x1r[2]-x1r[1])*(Xo1[Xo1>VolRange[1]]-x1r[1]),xlab='',ylab='',col='red',xlim=c(0,3),ylim=x1r,pch=4,xaxt='n',yaxt='n')
    
    par(mar=c(6.1,0,0,4.1))
    boxplot(X2,yaxt='n',ylim=x2r,xlim=c(0,2),col='lightblue',border='blue')
    grid(ny=NULL,nx=0)
    #segments(2,HSMX2,2,x2r[2],col='red')
    #par(new=TRUE);plot(x=c(2),y=c(HSMX2,x2r[2]),xlim=c(0,3),ylim=x2r,pch=c(10,24),xaxt='n',yaxt='n',ylab='',col='red',xlab='')
    axis(4)
    axis(1,at=c(1),label=c('State Data'),las=2)
    par(new=TRUE);plot(x=rep(1,length(Xo2[Xo2<VolRange[2]])),y=Xo2[Xo2<VolRange[2]],xlab='',ylab='',col='red',xlim=c(0,2),ylim=x2r,pch=4,xaxt='n',yaxt='n')
    
    title(paste(Title,'Predictor vs Predictor Plot'),outer=TRUE,line=-2)
    layout(1)
    par(mar=c(5,4,4,2)+0.1)
  }
  FType = toString(unique(D2$FType)[1])
  AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
  AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U6U')
  if( LogScale){SaveImage(paste(Title,'1 XXL'),'png')}
  if(!LogScale){SaveImage(paste(Title,'1 XX') ,'png')}
  
  if(FType %in% AllInt){PlotXXI (D2,D1,VolRange,HSMDefaults,Title)}
  if(FType %in% AllSeg){PlotXXS (D2,D1,VolRange,HSMDefaults,Title)}
  dev.off()
  #if( LogScale & FType %in% AllInt){PlotXXIL(D2,D1,VolRange,HSMDefaults,Title)}
  #if(!LogScale & FType %in% AllInt){PlotXXI (D2,D1,VolRange,HSMDefaults,Title)}
  #if( LogScale & FType %in% AllSeg){PlotXXSL(D2,D1,VolRange,HSMDefaults,Title)}
  #if(!LogScale & FType %in% AllSeg){PlotXXS (D2,D1,VolRange,HSMDefaults,Title)}
}


In [9]:
Plot2D        = function(Type,MD,LogScale = FALSE, Title = NULL,HSMCoef = NULL,Plot = TRUE){
  Plot2DInt = function(Type,MD,LogScale,Title,HSMCoef,Plot){
    P1Int = function(MD,LogScale,Title,HSMCoef){
      NumOfLines = 3
      M = MD[[1]]
      D = MD[[2]]
      Color = c('green4','blue','red')
      l = NULL
      
      x1rL = range(    D$AADT_Major )
      x1r  = range(exp(D$AADT_Major))
      x1L  = seq(x1rL[1],x1rL[2], by=(x1rL[2]-x1rL[1])/500)  
      x1   = seq(x1r [1],x1r [2], by=(x1r [2]-x1r [1])/500)  
      
      x2rL = range(    D$AADT_Minor )
      x2r  = range(exp(D$AADT_Minor))
      x2L  = seq(x2rL[1],x2rL[2],by=(x2rL[2]-x2rL[1])/NumOfLines)
      x2   = seq(x2r [1],x2r [2],by=(x2r [2]-x2r [1])/NumOfLines)
      
      yr = range(D$TOT_OC)
      for (i in 1:(length(x2)-1)){
        d = D[which(D$AADT_Minor >=x2L[i] & D$AADT_Minor <x2L[i+1]),]
        
        if( LogScale){plot(d$TOT_OC~    d$AADT_Major ,col=Color[i],xlab='AADT Major (Log Scale)',ylab='Total Observed Crashes',xlim=x1rL,ylim=yr,main=Title,pch=19)}
        if(!LogScale){plot(d$TOT_OC~exp(d$AADT_Major),col=Color[i],xlab='AADT Major'            ,ylab='Total Observed Crashes',xlim=x1r ,ylim=yr,main=Title,pch=19)}
        
        x2mL = (x2L[i]+x2L[i+1])/2
        x2m  = (x2 [i]+x2 [i+1])/2
        
        if( LogScale){lines(x1L,GLMPredict(M,data.frame('AADT_Major'=x1L    ,'AADT_Minor'=x2mL)),col=Color[i])}
        if(!LogScale){lines(x1 ,GLMPredict(M,data.frame('AADT_Major'=log(x1),'AADT_Minor'=x2mL)),col=Color[i])}
        
        #if(!sum(HSMCoef[1:3])==0){
        #  if( LogScale){lines(x1L,HSMPredict(M,data.frame('AADT_Major'=x1L    ,'AADT_Minor'=x2mL,'FType'=toString(unique(D$FType))),HSMCoef[4][[1]]),col=Color[i],lty=2)}
        #  if(!LogScale){lines(x1 ,HSMPredict(M,data.frame('AADT_Major'=log(x1),'AADT_Minor'=x2mL,'FType'=toString(unique(D$FType))),HSMCoef[4][[1]]),col=Color[i],lty=2)}
        #}
        l = c(l,paste('AADT Minor =~',round(x2m)))
        par(new='TRUE')
      }
      n = length(D[,1])
      legend(legend = l, 'topleft',text.col=Color)
      legend(legend=c(paste('Observed Data (n = ',n,')',sep=''),
                      paste('State Specific SPF (k = ',round(summary(MD[[1]])$dispers*1000)/1000,')',sep='')), 
                      'topright',
                      col='black',
                      lty=c(NA,1,2,NA),
                      pch=c(19,NA,NA,NA))
    }
    P2Int = function(MD,HSMCoef){
      Table = CoefSummary(MD)
      
      yr = range(Table$X.Intercept..UB,Table$X.Intercept..LB)
      if(length(MD[[1]]$coef)==3){
        xl = substitute(atop(A == C),list(C=round(Table$X.Intercept..Es*1000)/1000))
        yl = expression(N[spf] == exp(A + B ~ ln(AADT[Major]) + C ~ ln(AADT[Minor])))
        yl2 = ''
      }
      else{
        xl = substitute(atop(A + beta[i] * X[0][i] == B,HSM(A == D)),list(B=round(Table$X.Intercept..Es*1000)/1000,D=HSMCoef[1]))
        yl = expression(N[spf] == exp(A + B ~ ln(AADT[Major]) + C ~ ln(AADT[Minor]) + beta[i] * X[i]))
        Pred = NULL
        for(p in rownames(data.frame(summary(MD[[1]])$coef))){
          if(!(p %in% c('(Intercept)','AADT_Major','AADT_Minor'))){Pred = c(Pred,p)}}
        yl2 = substitute(X[i] ~ ~ A,list(A=paste(':',paste(Pred,collapse=' ,'))))
      }
      plot(Table$X.Intercept..Es,xlab=xl,ylab=yl,xaxt='n',col='blue',pch=20,cex=2,ylim=yr)
      segments(1,Table$X.Intercept..LB,1,Table$X.Intercept..UB,col='blue')
      par(new=TRUE);plot(Table$X.Intercept..LB,xlab='',ylab='',xaxt='n',col='blue',pch=25,cex=2,ylim=yr)
      par(new=TRUE);plot(Table$X.Intercept..UB,xlab='',ylab='',xaxt='n',col='blue',pch=24,cex=2,ylim=yr)
      #par(new=TRUE);plot(HSMCoef[1],xlab='',ylab='',xaxt='n',col='red' ,pch=1 ,cex=2,ylim=yr)
      
      par(new=FALSE)
      yr = range(Table$AADT_Major.UB,Table$AADT_Major.LB)
      plot(Table$AADT_Major.Es,xlab=substitute(atop(B == C),list(C=round(Table$AADT_Major.Es*1000)/1000)),ylab=yl2,xaxt='n',col='blue',pch=20,cex=2,ylim=yr);title('Coeff 95% CIs',line=1,font.main=1)
      segments(1,Table$AADT_Major.LB,1,Table$AADT_Major.UB,col='blue')
      par(new=TRUE);plot(Table$AADT_Major.LB,xlab='',ylab='',xaxt='n',col='blue',pch=25,cex=2,ylim=yr)
      par(new=TRUE);plot(Table$AADT_Major.UB,xlab='',ylab='',xaxt='n',col='blue',pch=24,cex=2,ylim=yr)
      
      
      par(new=FALSE)
      yr = range(Table$AADT_Minor.UB,Table$AADT_Minor.LB)
      plot(Table$AADT_Minor.Es,xlab=substitute(atop(C == D),list(D=round(Table$AADT_Minor.Es*1000)/1000)),ylab='',xaxt='n',col='blue',pch=20,cex=2,ylim=yr)
      segments(1,Table$AADT_Minor.LB,1,Table$AADT_Minor.UB,col='blue')
      par(new=TRUE);plot(Table$AADT_Minor.LB,xlab='',ylab='',xaxt='n',col='blue',pch=25,cex=2,ylim=yr)
      par(new=TRUE);plot(Table$AADT_Minor.UB,xlab='',ylab='',xaxt='n',col='blue',pch=24,cex=2,ylim=yr)
      
      title(paste(Title,'State Specific SPF'),outer= TRUE,line=-1.5)
    }
    SaveImage(paste(Title,'5 Plot2D'),'png')
    layout(matrix(c(1,2,3,4),1,4),width=c(5,1,1,1))
    P1Int(MD,LogScale,'',HSMCoef);title('Data and SPF Plot',line=1,font.main=1)
    P2Int(MD,HSMCoef)    
    dev.off()    
    #if(Plot){
    #  layout(matrix(c(1,2,3,4),1,),width=c(5,1,1,1))
    #  P1Int(MD,LogScale,'',HSMCoef);title('Data and SPF Plot',line=1,font.main=1)
    #  P2Int(MD,HSMCoef)    
    #}
    layout(1)
    par(new='FALSE')
  }
  Plot2DSeg = function(Type,MD,LogScale,Title,HSMCoef,Plot){
    P1Seg = function(MD,LogScale,Title,HSMCoef){
      NumOfLines = 3
      M = MD[[1]]
      D = MD[[2]]
      Color = c('green4','blue','red')
      l = NULL
      
      x1r  = range(exp(D$AADT))
      x1   = seq(x1r [1],x1r [2], by=(x1r [2]-x1r [1])/500)  
      
      x2r  = range(exp(D$Length))
      x2   = seq(x2r [1],x2r [2],by=(x2r [2]-x2r [1])/NumOfLines)
      
      yr = range(D$TOT_OC)
      for (i in 1:(length(x2)-1)){
        d = D[which(exp(D$Length) >=x2[i] & exp(D$Length) <x2[i+1]),]
        
        plot(d$TOT_OC~exp(d$AADT),col=Color[i],xlab='AADT'            ,ylab='Total Observed Crashes',xlim=x1r ,ylim=yr,main=Title,pch=19)
        
        x2m  = (x2 [i]+x2 [i+1])/2
        
        lines(x1 ,GLMPredict(M,data.frame('AADT'=log(x1),'Length'=log(x2m))),col=Color[i])
        
        #if(!sum(HSMCoef[1:3])==0){
        #  lines(x1 ,HSMPredict(M,data.frame('AADT'=log(x1),'Length'=log(x2m),'FType'=toString(unique(D$FType))),HSMCoef[4][[1]]),col=Color[i],lty=2)
        #}
        l = c(l,paste('Length =~',round(x2m*100)/100))
        par(new='TRUE')
      }
      n = length(D[,1])
      legend(legend = l, 'topleft',text.col=Color)
      legend(legend=c(paste('Observed Data (n = ',n,')',sep=''),
                      paste('State Specific SPF (k = 1/exp(',round(summary(MD[[1]])$dispers*1000)/1000,'+ln(L))',sep='')),
                      'topright',
                      col='black',
                      lty=c(NA,1),
                      pch=c(19,NA)
                     )
             
    }
    P2Seg = function(MD,HSMCoef){
      FType = toString(unique(MD[[2]]$FType)[1])
      Table = CoefSummary(MD)
      if(length(MD[[1]]$coef)==2){
        yl2 = ''
        yl  = expression(N[spf] == exp(A + B ~ ln(AADT) + Length))
      }
      else{
        Pred = NULL
        for(p in rownames(data.frame(summary(MD[[1]])$coef))){
          if(!(p %in% c('(Intercept)','AADT','Length'))){Pred = c(Pred,p)}}
        yl2 = substitute(X[i] ~ ~ A,list(A=paste(':',paste(Pred,collapse=' ,'))))
        yl  = substitute(N[spf] == exp(A + B ~ ln(AADT) + Length + beta[i] * X[i]))
      }
      if(FType %in% c('R2U','U2U','U3T','U4U','U4D','U5T','U6U')){
        yr1 = range(Table$X.Intercept..UB,Table$X.Intercept..LB)
        yr2 = range(Table$AADT.UB,Table$AADT.LB)
        if(length(MD[[1]]$coef)==2){xl1 = paste('A = ',round(Table$X.Intercept..Es*1000)/1000)}
        if(length(MD[[1]]$coef) >2){xl1 = substitute(A + beta[i] * X[0][i] == B,list(B=round(Table$X.Intercept..Es*1000)/1000))}
        xl2 = paste('B = ',round(Table$AADT.Es*1000)/1000)
      }
      if(FType %in% c('R4U','R4D')){
        yr1 = range(Table$X.Intercept..UB,Table$X.Intercept..LB)
        yr2 = range(Table$AADT.UB,Table$AADT.LB,HSMCoef[2])
        if(length(MD[[1]]$coef)==2){xl1 = substitute(atop(A == B),list(B=round(Table$X.Intercept..Es*1000)/1000))}
        if(length(MD[[1]]$coef) >2){xl1 = substitute(atop(A + beta[i] * X[0][i] == B),list(B=round(Table$X.Intercept..Es*1000)/1000))}
        xl2 = substitute(atop(B == C),list(C=round(Table$AADT.Es*1000)/1000))
      }
      plot(Table$X.Intercept..Es,xlab=xl1,ylab=yl,xaxt='n',col='blue',pch=20,cex=2,ylim=yr1)
      segments(1,Table$X.Intercept..LB,1,Table$X.Intercept..UB,col='blue')
      par(new=TRUE);plot(Table$X.Intercept..LB,xlab='',ylab='',xaxt='n',col='blue',pch=25,cex=2,ylim=yr1)
      par(new=TRUE);plot(Table$X.Intercept..UB,xlab='',ylab='',xaxt='n',col='blue',pch=24,cex=2,ylim=yr1)

      par(new=FALSE)
      plot(Table$AADT.Es,xlab=xl2,ylab=yl2,xaxt='n',col='blue',pch=20,cex=2,ylim=yr2);title('Coeff 95% CIs',line=1,font.main=1)
      segments(1,Table$AADT.LB,1,Table$AADT.UB,col='blue')
      par(new=TRUE);plot(Table$AADT.LB,xlab='',ylab='',xaxt='n',col='blue',pch=25,cex=2,ylim=yr2)
      par(new=TRUE);plot(Table$AADT.UB,xlab='',ylab='',xaxt='n',col='blue',pch=24,cex=2,ylim=yr2)
      title(paste(Title,'State Specific SPF'),outer= TRUE,line=-1.5)
    }
    SaveImage(paste(Title,'5 Plot2D'),'png')
    layout(matrix(c(1,2,3),1,3),width=c(4,1,1))
    P1Seg(MD,LogScale,'',HSMCoef);title('Data and SPF Plot',line=1,font.main=1)
    P2Seg(MD,HSMCoef)
    dev.off()    
    #if(Plot){
    #  layout(matrix(c(1,2,3),1,3),width=c(4,1,1))
    #  P1Seg(MD,LogScale,'',HSMCoef);title('Data and SPF Plot',line=1,font.main=1)
    #  P2Seg(MD,HSMCoef)      
    #}
    layout(1)
    par(new='FALSE')
  }
  #FType = toString(unique(MD[[2]]$FType)[1])
  AllInt = c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG','AllInt')
  AllSeg = c('R2U','R4U','R4D','U2U','U3T','U4U','U4D','U5T','U6U','AllSeg')
  if(Type %in% AllInt){Plot2DInt(Type,MD,LogScale,Title,HSMCoef,Plot)}
  if(Type %in% AllSeg){Plot2DSeg(Type,MD,LogScale,Title,HSMCoef,Plot)}
}


In [10]:
GLMModels     = function(D, MType = 'Base',FType = NULL){
  IntModels = function(D, MType, FType){
    HSMBaseModel      = function(D,Theta, FType){
      M = glm(TOT_OC ~ AADT_Major + AADT_Minor, data = D, family= negative.binomial(theta = Theta,link= "log"))
      return(list('Model' = M, 'Data' = D))
    }
    m = mean(D$TOT_OC)
    v = var(D$TOT_OC)
    Theta = m^2 / (v-m)
    if(Theta<0){Theta = 100000}
    if (MType == 'Base' ){return (HSMBaseModel     (D,Theta, FType))}
  }
  SegModels = function(D, MType, FType){
    HSMBaseModel      = function(D,Theta, FType){
      #M = glm(TOT_OC/exp(Length)/exp(AADT)/10^(-6)/365~1 , data = D, family= negative.binomial(theta = Theta,link= "log"))
      M = glm(TOT_OC ~ AADT + offset(Length), data = D, family= negative.binomial(theta = Theta,link= "log"))
      return(list('Model' = M, 'Data' = D))
    }
    m = mean(D$TOT_OC)
    v = var(D$TOT_OC)
    Theta = m^2 / (v-m)
    if(Theta<0){Theta = 100000}
    if (MType == 'Base' ){return (HSMBaseModel     (D,Theta, FType))}
    if (MType == 'Covar'){return (HSMCovariateModel(D,Theta, FType))}
    if (MType == 'Full' ){return (FullSegModel     (D,Theta, FType))}
    if (MType == 'All'  ){return (AllSegModel      (D,Theta, FType))}
  }
  if(is.null(FType)){FType = toString(unique(D$FType)[1])}
  AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG','AllInt','AllIntStp','AllIntSig'  )
  AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U','R4F','U4F','U6F','AllSeg')
  MDOut = NULL
  if(FType %in% AllInt){MDOut = IntModels(D,MType,FType)}
  if(FType %in% AllSeg){MDOut = SegModels(D,MType,FType)}
  return(MDOut)
}

In [11]:
CoefSummary   = function(MD,ConfLevel = 0.95){
  AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
  AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U')
  AllP = NULL;for(Type in c(AllInt,AllSeg)){AllP = c(AllP,ListPredictors(Type))};AllP = c('(Intercept)',unique(AllP))
  M = MD[[1]]
  D = MD[[2]]
  Type = toString(unique(D$FType)[1])
  AV = length(D[,1]);names(AV) = 'Sample Size'
  t  = qt((1+ConfLevel)/2,AV)
  P  = data.frame(summary(M)$coef)
  for(p in AllP){
    if(p %in% rownames(P)){
      AV = c(AV,P[p,1]            );names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.Es',sep=''))
      AV = c(AV,P[p,2]            );names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.SE'  ,sep=''))
      AV = c(AV,P[p,4]            );names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.Pv'  ,sep=''))
      AV = c(AV,P[p,1]-P[p,2]*t   );names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.LB'  ,sep=''))
      AV = c(AV,P[p,1]+P[p,2]*t   );names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.UB'  ,sep=''))
      AV = c(AV,abs(P[p,2]/P[p,1]));names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.CV' ,sep=''))
    }
    else{
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.Es',sep=''))
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.SE'  ,sep=''))
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.Pv'  ,sep=''))
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.LB'  ,sep=''))
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.UB'  ,sep=''))
      AV = c(AV,0);names(AV) = c(names(AV)[1:length(names(AV))-1],paste(p,'.CV' ,sep=''))
    }
  }
  Out = data.frame(t(data.frame(AV)))
  rownames(Out) = Type
  return(Out)
}  


In [12]:
ListPredictors  = function(Type = NULL){
  AllInt= c('AADT_Major','AADT_Minor')
  R3ST  = c(AllInt)
  R4ST  = c(AllInt)
  R4SG  = c(AllInt)
  RM3ST = c(AllInt)
  RM4ST = c(AllInt)
  RM4SG = c(AllInt)
  U3ST = c(AllInt)
  U4ST = c(AllInt)
  U3SG = c(AllInt)
  U4SG = c(AllInt)
  
  AllSeg = c('AADT','Length') 
  R2U = c(AllSeg)
  R4U = c(AllSeg)
  R4D = c(AllSeg)
  U2U = c(AllSeg)
  U3T = c(AllSeg)
  U4U = c(AllSeg)
  U4D = c(AllSeg)
  U6U = c(AllSeg)
  
  
  Out = list('R3ST'=R3ST,'R4ST'=R4ST,'R4SG'=R4SG,'RM3ST'=RM3ST,'RM4ST'=RM4ST,'RM4SG'=RM4SG,
             'U3ST'=U3ST,'U4ST'=U4ST,'U3SG'=U3SG,'U4SG'=U4SG,'AllInt'=AllInt,
             'R2U'=R2U,'R4D'=R4D,'R4U'=R4U,'U2U'=U2U,'U3T'=U3T,'U4D'=U4D,'U4U'=U4U,'U6U'=U6U,'AllSeg'=AllSeg)
  if(is.null(Type)){return(Out)}
  else(if(Type %in%  names(Out)){return(Out[Type][[1]])})
}


In [13]:
GLMPredict    = function(M,D,Type = NULL){
  Out = NULL
  if(is.null(Type)){Type = toString(unique(M$data$FType)[1])}
  P = ListHeaders(Type)
  V = ListBaseValues(Type)
  for(p in P){if(!(p %in% names(D)) & p %in% names(V)){D[p] = V[p]}}
  
  if(Type %in% c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG','AllInt')){
    Out = exp(predict.glm(M,D)) 
  }
  if(Type %in% c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U6U','R4F','U4F','U6F','AllSeg')){
    Out = exp(predict.glm(M,D))
  }
  return(Out)
}


In [14]:
CUREPlot      = function(MD,Title = ''){
  CUREPlot1 = function(DF1,Var = ''){
    COL = c('black','green4','green4','blue','blue','black')
    LTY = c(NA,1,2,1,2,NA)
    PCH = c(NA,20,NA,1,NA,NA)
    yr = range(DF1$LIM,DF1$CUMRES,-DF1$LIM)
    yr[2] = yr[2]*2
    plot(x=DF1$X,y=DF1$CUMRES,xlab=Var,ylab='Cumulative Residuals',col=COL[2],ylim=yr,pch=PCH[2])
    par(new=TRUE);plot(x=DF1$X,y=DF1$CUMRES,xlab='',ylab='',col=COL[2],ylim=yr,type='l',lty=LTY[2],xaxt='n',yaxt='n')
    par(new=TRUE);plot(y= DF1$LIM,x=DF1$X,xlab='',ylab='',ylim=yr,type='l',xaxt='n',yaxt='n',col=COL[3],lty=LTY[3])
    par(new=TRUE);plot(y=-DF1$LIM,x=DF1$X,xlab='',ylab='',ylim=yr,type='l',xaxt='n',yaxt='n',col=COL[3],lty=LTY[3])

    #par(new=TRUE);plot(x=DF2$X,y=DF2$CUMRES,xlab='',ylab='',col=COL[4],ylim=yr,pch=PCH[4],xaxt='n',yaxt='n')
    #par(new=TRUE);plot(x=DF2$X,y=DF2$CUMRES,xlab='',ylab='',col=COL[4],ylim=yr,type='l',lty=LTY[4],xaxt='n',yaxt='n')
    #par(new=TRUE);plot(y= DF2$LIM,x=DF2$X,xlab='',ylab='',ylim=yr,type='l',xaxt='n',yaxt='n',lty=LTY[5],col=COL[5])
    #par(new=TRUE);plot(y=-DF2$LIM,x=DF2$X,xlab='',ylab='',ylim=yr,type='l',xaxt='n',yaxt='n',lty=LTY[5],col=COL[5])
    grid()
    abline(h=0)
    #title(paste('CURE Plot for',Var))
  }
  CUREPlot2 = function(DF11,DF12,Var1,Var2,Title){
    COL = c('green4','green4','blue','blue')
    LTY = c(1,2,1,2)
    PCH = c(20,NA,1,NA)
    l1 = c(paste('Sample Size: ',length(DF11$X)))
    l2 = c('State SPF Fitted',expression(paste("±2",sigma,"* State SPF Limits")))
    
    layout(c(1,2),height=c(1.5,1))
    par(mar=c(4,4,3,1))
    CUREPlot1(DF11,Var1)    
    legend(legend=l1,'topleft')
    legend(legend=l2,lty=LTY,pch=PCH,'topright',text.col=COL,col=COL)
    par(mar=c(4,4,1,1))
    CUREPlot1(DF21,Var2)
    title(paste(Title,'CURE Plots'),outer = TRUE,line = -1.8)
    layout(1)
    par(mar=c(5,4,4,2)+.1)
  }
  DFGen = function(X,Y,Yhat){
    RES  = Y-Yhat
    DF = data.frame(X,RES)
    DF = DF[with(DF,order(X)),]
    CUMRES = NULL
    CUMRES2 = NULL
    CUMRES[1] = DF$RES[1]
    CUMRES2[1] = DF$RES[1]^2
    for(i in 2:length(DF$RES)){
      CUMRES[i] = CUMRES[i-1] + DF$RES[i]
      CUMRES2[i] = CUMRES2[i-1] + DF$RES[i]^2
    }
    DF['CUMRES'] = CUMRES
    DF['CUMRES2'] = CUMRES2
    VAR = NULL
    for(i in 1:length(DF$RES)){
      VAR[i] = CUMRES2[i] *(1-CUMRES2[i]/max(CUMRES2))
    }
    DF['LIM'] = 2 * sqrt(VAR)
    return (DF)
  }
  FType = toString(unique(MD[[2]]$FType)[1])
  AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
  if(FType %in% AllInt){
    DF11 = DFGen(exp(MD[[3]]$AADT_Major),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    DF21 = DFGen(exp(MD[[3]]$AADT_Minor),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    Var1 = 'AADT Major'
    Var2 = 'AADT Minor'
  }
  if(FType == 'R2U'){
    DF11 = DFGen(exp(MD[[3]]$AADT),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    DF21 = DFGen(exp(MD[[3]]$Length),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    Var1 = 'AADT'
    Var2 = 'Length'
  }  
  if(FType %in% c('R4U','R4D','U2U','U3T','U4U','U4D','U6U','R4F','U4F','U6F')){
    DF11 = DFGen(exp(MD[[3]]$AADT),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    DF21 = DFGen(exp(MD[[3]]$Length),MD[[3]]$TOT_OC,MD[[3]]$StateFitted)
    Var1 = 'AADT'
    Var2 = 'Length'
  }  
  
  SaveImage(paste(Title,'7 CURE'),'png')
  CUREPlot2(DF11,DF21,Var1,Var2,Title)
  dev.off()
  #CUREPlot2(DF11,DF21,Var1,Var2,Title)
}


In [15]:
ListHeaders     = function(Type){
  Out = NULL
  IntTypes = c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG','AllInt','AllIntSig','AllIntStp')
  SegTypes = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U6U','AllSeg')
  
  AllIntL = c('FID','TOT_OC','TOT_PC','AADT_Major','AADT_Minor','FType')
  AllSegL = c('FID','TOT_OC','TOT_PC','AADT'      ,'Length'    ,'FType')
  
  if(Type %in% c(IntTypes)){return(AllIntL)}
  if(Type %in% c(SegTypes)){return(AllSegL)}
}
ListBaseValues  = function(Type){
  D = data.frame()
  return(D)
}


In [16]:
HSMValues     = function(Type,CalibrationFactors = 1){
  IntTypes      = c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG','R2U' ,'R4U' ,'R4D' ,'U2U' ,'U3T' ,'U4U' ,'U4D' ,'U5T' ,'AllInt' , 'AllSeg')
  HSM_A         = c(-9.86 ,-8.56 ,-5.13 ,-12.526,-10.008,-7.182 ,-13.36 ,-8.9   ,-12.13 ,-10.99 ,-0.312,-9.653,-9.205,-15.22,-12.4 ,-11.63,-12.34,-9.7  ,0        ,0        )
  HSM_B         = c(0.79  ,0.6   ,0.6   ,1.204  ,0.848  ,0.722  ,1.11   ,0.82   ,1.11   ,1.07   ,0     ,1.176 ,1.049 ,1.68  ,1.41  ,1.33  ,1.36  ,1.17  ,0        ,0        )   
  HSM_C         = c(0.49  ,0.61  ,0.2   ,0.236  ,0.448  ,0.337  ,0.41   ,0.25   ,0.26   ,0.23   ,0     ,0     ,0     ,0     ,0     ,0     ,0     ,0     ,0        ,0        )
  AADT_Major_UB = c(19500 ,14700 ,25200 ,78300  ,78300  ,43500  ,45700  ,46800  ,58100  ,67700  ,17800 ,33200 ,89300 ,32600 ,32900 ,40100 ,66000 ,53800 ,78300    ,89300    )
  AADT_Minor_UB = c(4300  ,3500  ,12500 ,23000  ,7400   ,18500  ,9300   ,5900   ,16400  ,33400  ,0     ,0     ,0     ,0     ,0     ,0     ,0     ,0     ,33400    ,0        )
  Length_LB     = c(0     ,0     ,0     ,0      ,0      ,0      ,0      ,0      ,0      ,0      ,0.1   ,0.1   ,0.1   ,0.1   ,0.1   ,0.1   ,0.1   ,0.1   ,0        ,0.1      )
  k             = c(0.54  ,0.24  ,0.11  ,0.46   ,0.494  ,0.277  ,1.14   ,0.65   ,0.36   ,0.36   ,0.236 ,1.675 ,1.549 ,0.84  ,0.66  ,1.01  ,1.32  ,0.81  ,0        ,0        )
  SC_Cal        = CalibrationFactors
  HSM_D         = c(09    ,0     ,0     ,0      ,0      ,0      ,0      ,0      ,0      ,0      ,0     ,0     ,0     ,-5.47 ,-5.74 ,-7.99 ,-5.05 ,-4.82 ,0        ,0        )
  HSM_E         = c(0     ,0     ,0     ,0      ,0      ,0      ,0      ,0      ,0      ,0      ,0     ,0     ,0     ,0.56  ,0.54  ,0.81  ,0.47  ,0.54  ,0        ,0        )   
  
  D = data.frame('A'=HSM_A, 'B'=HSM_B, 'C'=HSM_C, 'C_Factor' = SC_Cal, 'AADT_Major_UB' = AADT_Major_UB,'AADT_Minor_UB' = AADT_Minor_UB,'Length_LB' = Length_LB,k,'D'=HSM_D,'E'=HSM_E)
  rownames(D) = IntTypes
  return(D[Type,])
}

In [17]:
Outliers      = function(MD, Plot = FALSE, Title = '', Max_Out = c(0,0,0)){
  OutliersInt   = function(MD, Plot, Title, Max_Out){
    FindIndex = function(Vector,Names){
      Out_Index = which(names(Vector) %in% Names)
      names(Out_Index) = Names
      return(Out_Index)
    }
    MaxOut_Cooks = 5
    MaxOut_JRes  = 5
    MaxOut_Lev   = 5
    if(length(Max_Out)>=1){MaxOut_Cooks = Max_Out[1]} 
    if(length(Max_Out)>=2){MaxOut_JRes  = Max_Out[2]} 
    if(length(Max_Out)>=3){MaxOut_Lev   = Max_Out[3]} 
    
    M = MD[[1]]
    D = MD[[2]]
    
    # Cooks Distance
    Cooks      = cooks.distance(M)
    names(Cooks) = rownames(D)
    #Threshold  = CooksRange[1] + (CooksRange[2]-CooksRange[1])*2/3
    #Out        = Cooks[Cooks>Threshold]
    #if(length(Out)>MaxOut_Cooks){
    #  Out_Order = Out[order(-Out)]
    #  Out = Out[names(Out_Order[1:MaxOut])]
    #}
    Cooks_Order = Cooks[order(-Cooks)]
    Out = Cooks_Order[1:MaxOut_Cooks]  
    if(MaxOut_Cooks<1){Out = NULL}
    Out_Index = FindIndex(Cooks,names(Out))
    Out_Cook    = strtoi(names(Out_Index))
    names(Out_Cook) = rep('Cooks Distance',length(Out_Cook))
    
    # Jackknife Residuals
    JRes = rstudent(M)
    names(JRes) = rownames(D)
    JRes_Order = JRes[order(-abs(JRes))]
    Out = JRes_Order[1:MaxOut_JRes]
    if(MaxOut_JRes<1){Out = NULL}
    Out_Index = FindIndex(JRes,names(Out))
    Out_JRes    = strtoi(names(Out_Index))
    names(Out_JRes) = rep('Jackknife Residuals',length(Out_JRes))
    
    # Leverage
    X = model.matrix(M)
    Lev = hat(X)
    names(Lev) = rownames(D)
    Lev_Order = Lev[order(-abs(Lev))]
    Out = Lev_Order[1:MaxOut_Lev]
    if(MaxOut_Lev<1){Out = NULL}
    Out_Index = FindIndex(Lev,names(Out))
    Out_Lev    = strtoi(names(Out_Index))
    names(Out_Lev) = rep('X Leverage',length(Out_Lev))
    
    Out_All = c(Out_Cook, Out_JRes, Out_Lev)
    
    # Plot
    PlotOutVec   = function(Vector,Names,Label='',Title = '',Color='red'){
      Out_Index = FindIndex(Vector,Names)
      Outliers = Vector[names(Vector) %in% Names]
      xr = c(1,length(Vector))
      yr = range(Vector)
      plot(Vector,xlab = 'Index', ylab=Label,xlim=xr,ylim=yr,main = Title,col='blue')
      #segments(1:length(Vector),0,1:length(Vector),Vector)
      par(new='TRUE')
      plot(x=Out_Index,y=Outliers,col=Color,xlim=xr,ylim=yr,xlab = 'Index', ylab=Label,pch=4)
      par(new='FALSE')
    }
    PlotOutXX    = function(D,Names,Title = '',Color){
      xr = range(D$AADT_Major)
      yr = range(D$AADT_Minor)
      d = D[rownames(D) %in% Names,]
      plot(D$AADT_Minor~D$AADT_Major,xlab = 'AADT Major (LogScale)', ylab='AADT Minor (LogScale)',xlim=xr,ylim=yr,main = Title,col='blue')
      #segments(1:length(Vector),0,1:length(Vector),Vector)
      par(new='TRUE')
      plot(d$AADT_Minor~d$AADT_Major,col=Color,xlim=xr,ylim=yr,xlab = 'AADT Major (LogScale)', ylab='AADT Minor (LogScale)',pch=4)
      par(new='FALSE')
    }
    PlotOutliers = function(D,Out_Names,Title='',Color){
      xr = range(D$AADT_Major)
      yr = range(D$TOT_OC)
      plot(D$TOT_OC ~ D$AADT_Major, xlim=xr, ylim=yr,col=Color[1], xlab='AADT Major (Log Scale)', ylab='TOT Observed Crashes',,main = Title)
      l = paste('Observations (n=',length(D[,1]),')',sep='')
      Series = names(Out_Names)[!duplicated(names(Out_Names))]
      for(i in 1:length(Series)){
        par(new='TRUE')
        d = D[paste(Out_Names[names(Out_Names)==Series[i]]),]
        plot(d$TOT_OC ~ d$AADT_Major, xlim=xr, ylim=yr, col=Color[i+1], xlab='', ylab='',pch=4,cex=1.5)
        l = c(l,paste('Outliers by ',Series[i],' (n=',length(d[,1]),')',sep=''))
      }
      legend(legend = l, 'topleft',text.col=Color,pch=c(1,4,4,4),col=Color)
      par(new='FALSE')
    }
    Color = c('blue','darkorange3','green4','red')
    SaveImage(paste(Title,'2 Outliers'),'png');
    layout(matrix(c(1,1,1,2,2,2,5,5,5,5,3,3,5,5,5,5,4,4),3,6,byrow=TRUE))
    PlotOutVec  (Cooks,Out_Cook,'Cooks Dist','',Color[2]);title('Cooks Distance'              ,line=1,font.main=1)
    PlotOutVec  (JRes ,Out_JRes,'J Res'     ,'',Color[3]);title('Jackknife Residuals'         ,line=1,font.main=1)  
    PlotOutXX   (D    ,Out_Lev ,''             ,Color[4]);title('Predictor vs Predictor Plot' ,line=1,font.main=1)
    PlotOutVec  (Lev  ,Out_Lev ,'X Lev'     ,'',Color[4]);title('X Leverages'                 ,line=1,font.main=1)  
    PlotOutliers(D    ,Out_All ,''             ,Color   );title('Predictor vs Prediction Plot'  ,line=1,font.main=1)
    title(paste(Title,'Outliers'),outer=TRUE,line=-2)
    dev.off()    
    
    #if(Plot){
    #  layout(matrix(c(1,1,1,2,2,2,5,5,5,5,3,3,5,5,5,5,4,4),3,6,byrow=TRUE))
    #  PlotOutVec  (Cooks,Out_Cook,'Cooks Dist','',Color[2]);title('Cooks Distance'      ,line=1,font.main=1)
    #  PlotOutVec  (JRes ,Out_JRes,'J Res'     ,'',Color[3]);title('Jackknife Residuals' ,line=1,font.main=1)  
    #  PlotOutXX   (D    ,Out_Lev ,''             ,Color[4]);title('Predictor vs Predictor Plot'             ,line=1,font.main=1)
    #  PlotOutVec  (Lev  ,Out_Lev ,'X Lev'     ,'',Color[4]);title('X Leverages'         ,line=1,font.main=1)  
    #  PlotOutliers(D    ,Out_All ,''             ,Color   );title('Predictor vs Prediction Plot'             ,line=1,font.main=1)
    #  title(paste(Title,'Outliers'),outer=TRUE,line=-2)
    #}
    
    D_Out = D[paste(Out_All),]
    D_Out['Outlier'] = names(c(Out_Cook, Out_JRes, Out_Lev))
    #print(D_Out)
    #layout(1)
    return(D_Out)
  }
  OutliersSeg   = function(MD, Plot, Title, Max_Out){
    FindIndex = function(Vector,Names){
      Out_Index = which(names(Vector) %in% Names)
      names(Out_Index) = Names
      return(Out_Index)
    }
    MaxOut_Cooks = 5
    MaxOut_JRes  = 5
    MaxOut_Lev   = 5
    if(length(Max_Out)>=1){MaxOut_Cooks = Max_Out[1]} 
    if(length(Max_Out)>=2){MaxOut_JRes  = Max_Out[2]} 
    if(length(Max_Out)>=3){MaxOut_Lev   = Max_Out[3]} 
    
    M = MD[[1]]
    D = MD[[2]]
    
    # Cooks Distance
    Cooks      = cooks.distance(M)
    names(Cooks) = rownames(D)
    Cooks_Order = Cooks[order(-Cooks)]
    Out = Cooks_Order[1:MaxOut_Cooks]  
    if(MaxOut_Cooks<1){Out = NULL}
    Out_Index = FindIndex(Cooks,names(Out))
    Out_Cook    = strtoi(names(Out_Index))
    names(Out_Cook) = rep('Cooks Distance',length(Out_Cook))
    
    # Jackknife Residuals
    JRes = rstudent(M)
    names(JRes) = rownames(D)
    JRes_Order = JRes[order(-abs(JRes))]
    Out = JRes_Order[1:MaxOut_JRes]
    if(MaxOut_JRes<1){Out = NULL}
    Out_Index = FindIndex(JRes,names(Out))
    Out_JRes    = strtoi(names(Out_Index))
    names(Out_JRes) = rep('Jackknife Residuals',length(Out_JRes))
    
    # Leverage
    X = model.matrix(M)
    Lev = hat(X)
    names(Lev) = rownames(D)
    Lev_Order = Lev[order(-abs(Lev))]
    Out = Lev_Order[1:MaxOut_Lev]
    if(MaxOut_Lev<1){Out = NULL}
    Out_Index = FindIndex(Lev,names(Out))
    Out_Lev    = strtoi(names(Out_Index))
    names(Out_Lev) = rep('X Leverage',length(Out_Lev))
    
    Out_All = c(Out_Cook, Out_JRes, Out_Lev)
    
    # Plot
    PlotOutVec   = function(Vector,Names,Label='',Title = '',Color='red'){
      Out_Index = FindIndex(Vector,Names)
      Outliers = Vector[names(Vector) %in% Names]
      xr = c(1,length(Vector))
      yr = range(Vector)
      if(yr[1]==yr[2]){yr[2]=1.5*yr[1]}
      plot(Vector,xlab = 'Index', ylab=Label,xlim=xr,ylim=yr,main = Title,col='blue')
      #segments(1:length(Vector),0,1:length(Vector),Vector)
      par(new='TRUE')
      plot(x=Out_Index,y=Outliers,col=Color,xlim=xr,ylim=yr,xlab = 'Index', ylab=Label,pch=4)
      par(new='FALSE')
    }
    PlotOutXX    = function(D,Names,Title = '',Color){
      xr = range(D$AADT)
      yr = range(exp(D$Length))
      d = D[rownames(D) %in% Names,]
      plot(exp(D$Length)~D$AADT,xlab = 'AADT (LogScale)', ylab='Length (LogScale)',xlim=xr,ylim=yr,main = Title,col='blue')
      #segments(1:length(Vector),0,1:length(Vector),Vector)
      par(new='TRUE')
      plot(d$Length~d$AADT,col=Color,xlim=xr,ylim=yr,xlab = 'AADT (LogScale)', ylab='Length (LogScale)',pch=4)
      par(new='FALSE')
    }
    PlotOutliers = function(D,Out_Names,Title='',Color){
      xr = range(D$AADT)
      yr = range(D$TOT_OC)
      plot(D$TOT_OC ~ D$AADT, xlim=xr, ylim=yr,col=Color[1], xlab='AADT (Log Scale)', ylab='Total Observed Crashes',,main = Title)
      l = paste('Observations (n=',length(D[,1]),')',sep='')
      Series = names(Out_Names)[!duplicated(names(Out_Names))]
      for(i in 1:length(Series)){
        par(new='TRUE')
        d = D[paste(Out_Names[names(Out_Names)==Series[i]]),]
        plot(d$TOT_OC ~ d$AADT, xlim=xr, ylim=yr, col=Color[i+1], xlab='', ylab='',pch=4,cex=1.5)
        l = c(l,paste('Outliers by ',Series[i],' (n=',length(d[,1]),')',sep=''))
      }
      legend(legend = l, 'topleft',text.col=Color,pch=c(1,4,4,4),col=Color)
      par(new='FALSE')
    }
    Color = c('blue','darkorange3','green4','red')
    SaveImage(paste(Title,'2 Outliers'),'png');
    layout(matrix(c(1,1,1,2,2,2,5,5,5,5,3,3,5,5,5,5,4,4),3,6,byrow=TRUE))
    PlotOutVec  (Cooks,Out_Cook,'Cooks Dist','',Color[2]);title('Cooks Distance'              ,line=1,font.main=1)
    PlotOutVec  (JRes ,Out_JRes,'J Res'     ,'',Color[3]);title('Jackknife Residuals'         ,line=1,font.main=1)  
    PlotOutXX   (D    ,Out_Lev ,''             ,Color[4]);title('Predictor vs Predictor Plot' ,line=1,font.main=1)
    PlotOutVec  (Lev  ,Out_Lev ,'X Lev'     ,'',Color[4]);title('X Leverages'                 ,line=1,font.main=1)  
    PlotOutliers(D    ,Out_All ,''             ,Color   );title('Predictor vs Variable Plot'  ,line=1,font.main=1)
    title(paste(Title,'Outliers'),outer=TRUE,line=-2)
    dev.off()    
    
    #if(Plot==1){
    #  layout(matrix(c(1,1,1,2,2,2,5,5,5,5,3,3,5,5,5,5,4,4),3,6,byrow=TRUE))
    #  PlotOutVec  (Cooks,Out_Cook,'Cooks Dist','',Color[2]);title('Cooks Distance'      ,line=1,font.main=1)
    #  PlotOutVec  (JRes ,Out_JRes,'J Res'     ,'',Color[3]);title('Jackknife Residuals' ,line=1,font.main=1)  
    #  PlotOutXX   (D    ,Out_Lev ,''             ,Color[4]);title('Predictor vs Predictor Plot'             ,line=1,font.main=1)
    #  PlotOutVec  (Lev  ,Out_Lev ,'X Lev'     ,'',Color[4]);title('X Leverages'         ,line=1,font.main=1)  
    #  PlotOutliers(D    ,Out_All ,''             ,Color   );title('Predictor vs Prediction Plot'             ,line=1,font.main=1)
    #  title(paste(Title,'Outliers'),outer=TRUE,line=-2)
    #}
    
    D_Out = D[paste(Out_All),]
    D_Out['Outlier'] = names(c(Out_Cook, Out_JRes, Out_Lev))
    #print(D_Out)
    #layout(1)
    return(D_Out)
  }
  FType = toString(unique(MD[[2]]$FType)[1])
  if(Title == ''){Title = FType}
  AllInt = c('R3ST'  ,'R4ST'  ,'R4SG'  ,'RM3ST'  ,'RM4ST'  ,'RM4SG'  ,'U3ST'  ,'U4ST'  ,'U3SG'  ,'U4SG'  )
  AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U')
  MDOut = NULL
  if(FType %in% AllInt){MDOut = OutliersInt(MD,Plot,Title,Max_Out)}
  if(FType %in% AllSeg){MDOut = OutliersSeg(MD,Plot,Title,Max_Out)}
  return(MDOut)
}


In [18]:
HSMPredict    = function(M,D,CF=1){
  Out = NULL
  AllInt = c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG')
  AllSeg = c('R2U','R4D','R4U','U2U','U3T','U4U','U4D','U5T','U6U')
  HSMC = NULL;for(t in c(AllInt,AllSeg)){HSMC=rbind(HSMC,HSMValues(t,CF))}
  for(i in 1:length(D[,1])){
    d = D[i,]
    Type = toString(d$FType)
    if(Type %in% c('R3ST','R4ST','R4SG','RM3ST','RM4ST','RM4SG','U3ST','U4ST','U3SG','U4SG')){
      Out = c(Out,HSMC[Type,4][[1]]*exp(HSMC[Type,1][[1]] + HSMC[Type,2][[1]]*d$AADT_Major + HSMC[Type,3][[1]]*d$AADT_Minor))
    }
    if(Type == 'R2U'){
      Out = c(Out,HSMC[Type,4][[1]]*(exp(d$AADT)*exp(d$Length)*365*10^(-6)*exp(HSMC[Type,1])[[1]]))  
    }
    if(Type %in% c('R4D','R4U')){
      Out = c(Out,HSMC[Type,4][[1]]*exp(d$Length)*exp(HSMC[Type,1][[1]]+HSMC[Type,2][[1]]*d$AADT))
    }
    if(Type %in% c('U2U','U3T','U4U','U4D','U5T','U6U')){
      Out = c(Out,HSMC[Type,4][[1]]*exp(d$Length)*exp(HSMC[Type,1][[1]]+HSMC[Type,2][[1]]*d$AADT)
              +HSMC[Type,4][[1]]*exp(d$Length)*exp(HSMC[Type,9][[1]]+HSMC[Type,10][[1]]*d$AADT))
    }
  }
  return(Out)
}
